# 🌌 Heston IV Surfaces (Carr-Madan)

Notebook dédié à la construction des surfaces d'IV (calls & puts) et des heatmaps de prix analytiques :
1. Téléchargement CBOE
2. Calibration Heston ciblée (Carr-Madan)
3. Calcul des surfaces et heatmaps (pas de Monte Carlo)

## Pré-requis
- `pip install -r requirements.txt`
- Accès réseau pour télécharger les options (CBOE delayed)
- GPU optionnel (CPU suffit)


In [23]:
from __future__ import annotations

import math
import requests
import re
from pathlib import Path
from typing import Callable

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import torch
from IPython.display import display
from scipy.interpolate import griddata

from heston_torch import HestonParams, carr_madan_call_torch

torch.set_default_dtype(torch.float64)
DEVICE = torch.device("cpu")
MIN_IV_MATURITY = 0.1

## Téléchargement via CBOE delayed

In [24]:
def download_options_cboe(symbol: str, option_type: str, save_path: str) -> tuple[pd.DataFrame, float]:
    url = f"https://cdn.cboe.com/api/global/delayed_quotes/options/{symbol.upper()}.json"
    resp = requests.get(url, timeout=15)
    resp.raise_for_status()
    payload = resp.json()
    data = payload.get("data", {})
    options = data.get("options", [])
    spot = float(data.get("current_price") or data.get("close") or np.nan)
    now = pd.Timestamp.utcnow().tz_localize(None)
    pattern = re.compile(rf"^{symbol.upper()}(?P<expiry>\d{{6}})(?P<cp>[CP])(?P<strike>\d+)$")

    rows = []
    for opt in options:
        m = pattern.match(opt.get("option", ""))
        if not m:
            continue
        cp = m.group("cp")
        if (option_type == "call" and cp != "C") or (option_type == "put" and cp != "P"):
            continue
        expiry_dt = pd.to_datetime(m.group("expiry"), format="%y%m%d")
        T = (expiry_dt - now).total_seconds() / (365.0 * 24 * 3600)
        if T <= 0:
            continue
        T = round(T, 2)
        strike = int(m.group("strike")) / 1000.0
        bid = float(opt.get("bid") or 0.0)
        ask = float(opt.get("ask") or 0.0)
        last = float(opt.get("last_trade_price") or 0.0)
        mid = np.nan
        if bid > 0 and ask > 0:
            mid = 0.5 * (bid + ask)
        elif last > 0:
            mid = last
        if np.isnan(mid) or mid <= 0:
            continue
        mid = round(mid, 2)
        iv_val = opt.get("iv", np.nan)
        iv_val = float(iv_val) if iv_val not in (None, "") else np.nan
        rows.append({
            "S0": spot,
            "K": strike,
            "T": T,
            ("C_mkt" if option_type == "call" else "P_mkt"): mid,
            "iv_market": iv_val,
        })

    df = pd.DataFrame(rows)
    df = df[df["T"] > MIN_IV_MATURITY]
    df.to_csv(save_path, index=False)
    print(f"Sauvegardé {len(df)} lignes dans {save_path}")
    return df, spot

## Calibration Heston (Carr-Madan)

In [25]:
def prices_from_unconstrained(u, S0_t, K_t, T_t, r, q):
    params = HestonParams.from_unconstrained(u[0], u[1], u[2], u[3], u[4])
    prices = []
    for S0_i, K_i, T_i in zip(S0_t, K_t, T_t):
        prices.append(carr_madan_call_torch(S0_i, r, q, T_i, params, K_i))
    return torch.stack(prices)


def loss(u, S0_t, K_t, T_t, C_t, r, q, weights=None):
    model_prices = prices_from_unconstrained(u, S0_t, K_t, T_t, r, q)
    diff = model_prices - C_t
    if weights is not None:
        return 0.5 * (weights * diff**2).mean()
    return 0.5 * (diff**2).mean()


def calibrate_heston_nn(df, r, q, max_iters=300, lr=0.01, spot_override=None):
    df_clean = df.dropna(subset=["S0", "K", "T", "C_mkt"])
    df_clean = df_clean[(df_clean["T"] > MIN_IV_MATURITY) & (df_clean["C_mkt"] > 0.05)]
    df_clean = df_clean[df_clean.get("iv_market", 0) > 0]
    if df_clean.empty:
        raise ValueError("Aucun point pour la calibration")

    S0_ref = spot_override if spot_override is not None else float(df_clean["S0"].median())
    moneyness = df_clean["K"].values / S0_ref

    S0_t = torch.tensor(df_clean["S0"].values, dtype=torch.float64, device=DEVICE)
    K_t = torch.tensor(df_clean["K"].values, dtype=torch.float64, device=DEVICE)
    T_t = torch.tensor(df_clean["T"].values, dtype=torch.float64, device=DEVICE)
    C_t = torch.tensor(df_clean["C_mkt"].values, dtype=torch.float64, device=DEVICE)

    weights_np = 1.0 / (np.abs(moneyness - 1.0) + 1e-3)
    weights_np = np.clip(weights_np / weights_np.mean(), 0.5, 5.0)
    weights_t = torch.tensor(weights_np, dtype=torch.float64, device=DEVICE)

    u = torch.zeros(5, dtype=torch.float64, device=DEVICE, requires_grad=True)
    optimizer = torch.optim.Adam([u], lr=lr)

    for it in range(max_iters):
        optimizer.zero_grad()
        loss_val = loss(u, S0_t, K_t, T_t, C_t, r, q, weights=weights_t)
        loss_val.backward()
        optimizer.step()
        if (it + 1) % 50 == 0:
            print(f"Iter {it+1}/{max_iters} loss={loss_val.item():.6f}")

    return HestonParams.from_unconstrained(u[0], u[1], u[2], u[3], u[4])


def bs_call(S, K, T, r, sigma):
    if T <= 0 or sigma <= 0:
        return max(S - K, 0)
    d1 = (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    from scipy.stats import norm
    return S * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)


def bs_put(S, K, T, r, sigma):
    if T <= 0 or sigma <= 0:
        return max(K - S, 0)
    d1 = (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    from scipy.stats import norm
    return K * math.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)


def implied_vol(price, S, K, T, r, option_type="call"):
    if T < MIN_IV_MATURITY:
        return np.nan
    intrinsic = max(S - K, 0) if option_type == "call" else max(K - S, 0)
    if price <= intrinsic:
        return np.nan
    sigma = 0.3
    for _ in range(100):
        est = bs_call(S, K, T, r, sigma) if option_type == "call" else bs_put(S, K, T, r, sigma)
        diff = est - price
        if abs(diff) < 1e-6:
            return sigma
        d1 = (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
        from scipy.stats import norm
        vega = S * norm.pdf(d1) * math.sqrt(T)
        if vega < 1e-8:
            return np.nan
        sigma -= diff / vega
        if sigma <= 0:
            return np.nan
    return np.nan

## Paramètres

In [26]:
symbol = "SPY"
rf_rate = 0.02
div_yield = 0.0
span_mc = 20.0
step_strike = 1.0
n_maturities = 40
T_band = (0.82, 0.90)
calib_span = 20
max_iters = 1000
learning_rate = 0.005

## Step 1 — Téléchargement

In [27]:
calls_df, S0_ref = download_options_cboe(symbol, "call", "data/iv_calls_only.csv")
puts_df, _ = download_options_cboe(symbol, "put", "data/iv_puts_only.csv")
print(f"Calls: {len(calls_df)} | Puts: {len(puts_df)} | S0_ref={S0_ref:.2f}")

Sauvegardé 3243 lignes dans data/iv_calls_only.csv
Sauvegardé 3243 lignes dans data/iv_puts_only.csv
Calls: 3243 | Puts: 3243 | S0_ref=675.31


## Step 2 — Calibration

In [28]:
calib_slice = calls_df[
    (calls_df["T"].round(2).between(*T_band)) &
    (calls_df["K"].between(S0_ref - calib_span, S0_ref + calib_span)) &
    (calls_df["C_mkt"] > 0.05) &
    (calls_df["iv_market"] > 0)
]
print(f"Calibration sur {len(calib_slice)} points")
params_cm = calibrate_heston_nn(calib_slice, r=rf_rate, q=div_yield, max_iters=max_iters, lr=learning_rate, spot_override=S0_ref)
print({k: float(getattr(params_cm, k).detach()) for k in ['kappa','theta','sigma','rho','v0']})

Calibration sur 45 points
Iter 50/1000 loss=8491.782489
Iter 100/1000 loss=6024.805302
Iter 150/1000 loss=4028.331909
Iter 200/1000 loss=2547.014785
Iter 250/1000 loss=1563.666197
Iter 300/1000 loss=959.551025
Iter 350/1000 loss=598.001906
Iter 400/1000 loss=380.094181
Iter 450/1000 loss=246.043606
Iter 500/1000 loss=161.663970
Iter 550/1000 loss=107.429009
Iter 600/1000 loss=71.960342
Iter 650/1000 loss=48.448635
Iter 700/1000 loss=32.707175
Iter 750/1000 loss=22.096824
Iter 800/1000 loss=14.917258
Iter 850/1000 loss=10.052614
Iter 900/1000 loss=6.759473
Iter 950/1000 loss=4.536674
Iter 1000/1000 loss=3.043430
{'kappa': 0.08514720406793155, 'theta': 0.2838360321915019, 'sigma': 2.4833277685424804, 'rho': -0.7436896210233432, 'v0': 0.1772674897205345}


## Step 3 — IV Surfaces & Heatmaps analytiques

In [30]:
K_grid = np.arange(S0_ref - span_mc, S0_ref + span_mc + step_strike, step_strike)
T_grid = np.linspace(0.1, 2.0, n_maturities)
print(f"Grille K={len(K_grid)} | T={len(T_grid)}")

call_prices_cm = np.zeros((len(T_grid), len(K_grid)))
put_prices_cm = np.zeros_like(call_prices_cm)
Ks_t = torch.tensor(K_grid, dtype=torch.float64)

for i, T_val in enumerate(T_grid):
    call_vals = carr_madan_call_torch(S0_ref, rf_rate, div_yield, float(T_val), params_cm, Ks_t)
    discount = torch.exp(-torch.tensor(rf_rate * T_val, dtype=torch.float64))
    forward = torch.exp(-torch.tensor(div_yield * T_val, dtype=torch.float64))
    put_vals = call_vals - S0_ref * forward + Ks_t * discount
    call_prices_cm[i,:] = call_vals.detach().cpu().numpy()
    put_prices_cm[i,:] = put_vals.detach().cpu().numpy()

call_iv_cm = np.zeros_like(call_prices_cm)
put_iv_cm = np.zeros_like(put_prices_cm)
for i, T_val in enumerate(T_grid):
    for j, K_val in enumerate(K_grid):
        call_iv_cm[i,j] = implied_vol(call_prices_cm[i,j], S0_ref, K_val, T_val, rf_rate, "call")
        put_iv_cm[i,j] = implied_vol(put_prices_cm[i,j], S0_ref, K_val, T_val, rf_rate, "put")

KK_cm, TT_cm = np.meshgrid(K_grid, T_grid, indexing="xy")
fig_call = go.Figure(data=[go.Surface(x=KK_cm, y=TT_cm, z=call_iv_cm, colorscale='Viridis')])
fig_call.update_layout(title=f"IV Surface Calls (Carr-Madan) - {symbol}", scene=dict(xaxis_title='K', yaxis_title='T', zaxis_title='IV'), height=600)
fig_call.show()

fig_put = go.Figure(data=[go.Surface(x=KK_cm, y=TT_cm, z=put_iv_cm, colorscale='Viridis')])
fig_put.update_layout(title=f"IV Surface Puts (Carr-Madan) - {symbol}", scene=dict(xaxis_title='K', yaxis_title='T', zaxis_title='IV'), height=600)
fig_put.show()

# Surfaces marché recalculées

def build_market_surface(df, price_col, opt_type):
    df = df.dropna(subset=[price_col]).copy()
    df = df[(df["T"] >= MIN_IV_MATURITY) & (df[price_col] > 0)]
    df["iv_calc"] = df.apply(lambda row: implied_vol(row[price_col], S0_ref, row["K"], row["T"], rf_rate, opt_type), axis=1)
    df = df.dropna(subset=["iv_calc"])
    if len(df) < 5:
        return None
    surf = griddata(df[["K","T"]].to_numpy(), df["iv_calc"].to_numpy(), (KK_cm, TT_cm), method='linear')
    if surf is None or np.all(np.isnan(surf)):
        surf = griddata(df[["K","T"]].to_numpy(), df["iv_calc"].to_numpy(), (KK_cm, TT_cm), method='nearest')
    else:
        nan_mask = np.isnan(surf)
        if nan_mask.any():
            surf[nan_mask] = griddata(df[["K","T"]].to_numpy(), df["iv_calc"].to_numpy(), (KK_cm[nan_mask], TT_cm[nan_mask]), method='nearest')
    return surf

surf_call_mkt = build_market_surface(calls_df, "C_mkt", "call")
if surf_call_mkt is not None:
    fig_call_mkt = go.Figure(data=[go.Surface(x=KK_cm, y=TT_cm, z=surf_call_mkt, colorscale='Plasma')])
    fig_call_mkt.update_layout(title=f"IV Surface Calls Marché (recalc) - {symbol}", scene=dict(xaxis_title='K', yaxis_title='T', zaxis_title='IV'), height=600)
    fig_call_mkt.show()

surf_put_mkt = build_market_surface(puts_df, "P_mkt", "put")
if surf_put_mkt is not None:
    fig_put_mkt = go.Figure(data=[go.Surface(x=KK_cm, y=TT_cm, z=surf_put_mkt, colorscale='Plasma')])
    fig_put_mkt.update_layout(title=f"IV Surface Puts Marché (recalc) - {symbol}", scene=dict(xaxis_title='K', yaxis_title='T', zaxis_title='IV'), height=600)
    fig_put_mkt.show()

# Heatmaps prix analytiques
fig_heat_call = go.Figure(data=[go.Heatmap(z=call_prices_cm, x=K_grid, y=T_grid, colorscale='Viridis', colorbar=dict(title='Call'))])
fig_heat_call.update_layout(title=f"Heatmap Prix Calls (Carr-Madan) - {symbol}", xaxis_title='Strike K', yaxis_title='T')
fig_heat_call.show()

fig_heat_put = go.Figure(data=[go.Heatmap(z=put_prices_cm, x=K_grid, y=T_grid, colorscale='Viridis', colorbar=dict(title='Put'))])
fig_heat_put.update_layout(title=f"Heatmap Prix Puts (Carr-Madan) - {symbol}", xaxis_title='Strike K', yaxis_title='T')
fig_heat_put.show()


# Heatmaps prix marché (interp sur la même grille)
def build_market_price_grid(df, price_col):
    df = df.dropna(subset=[price_col]).copy()
    df = df[(df["T"] >= MIN_IV_MATURITY) & (df[price_col] > 0)]
    if len(df) < 5:
        return None
    pts = df[["K","T"]].to_numpy()
    vals = df[price_col].to_numpy()
    grid = griddata(pts, vals, (KK_cm, TT_cm), method='linear')
    if grid is None or np.all(np.isnan(grid)):
        grid = griddata(pts, vals, (KK_cm, TT_cm), method='nearest')
    else:
        nan_mask = np.isnan(grid)
        if nan_mask.any():
            grid[nan_mask] = griddata(pts, vals, (KK_cm[nan_mask], TT_cm[nan_mask]), method='nearest')
    return grid

market_call_grid = build_market_price_grid(calls_df, "C_mkt")
if market_call_grid is not None:
    fig_heat_call_mkt = go.Figure(data=[go.Heatmap(z=market_call_grid, x=K_grid, y=T_grid, colorscale='Plasma', colorbar=dict(title='C_mkt'))])
    fig_heat_call_mkt.update_layout(title=f"Heatmap Prix Calls (Marché) - {symbol}", xaxis_title='Strike K', yaxis_title='T (années)')
    fig_heat_call_mkt.show()

market_put_grid = build_market_price_grid(puts_df, "P_mkt")
if market_put_grid is not None:
    fig_heat_put_mkt = go.Figure(data=[go.Heatmap(z=market_put_grid, x=K_grid, y=T_grid, colorscale='Plasma', colorbar=dict(title='P_mkt'))])
    fig_heat_put_mkt.update_layout(title=f"Heatmap Prix Puts (Marché) - {symbol}", xaxis_title='Strike K', yaxis_title='T (années)')
    fig_heat_put_mkt.show()


Grille K=41 | T=40
